In [9]:
import os
import warnings
import numpy as np
import pandas as pd
import functions

from lambeq import BobcatParser

warnings.filterwarnings('ignore')
os.environ['TOKENIZERS_PARALLELISM'] = 'true'



In [49]:
## IMDB set

## Process data, split into training/test
imdb_data = pd.read_csv('IMDB Dataset.csv')
## My PC is weak as fuck so truncate Dataset
imdb_data = imdb_data[1:1000]
size = int(len(imdb_data)*0.8)
imdb_training = imdb_data.loc[:size].copy()
imdb_testing = imdb_data.loc[size:].copy()

## convert to numpy array
train, trainlabels = imdb_training.iloc[:,0].to_numpy(), imdb_training.iloc[:,1].to_numpy()
test, testlabels = imdb_testing.iloc[:,0].to_numpy(), imdb_testing.iloc[:,1].to_numpy()


In [79]:
## Twitter dataset
negative = np.loadtxt("processedNegative.csv", delimiter=",", dtype = str).reshape(-1,1)
neglabels = np.full(np.shape(negative),-1)
neutral = np.loadtxt("processedNeutral.csv", delimiter=",", dtype = str).reshape(-1,1)
neutlabels = np.full(np.shape(neutral),0)
positive = np.loadtxt("processedPositive.csv", delimiter=",", dtype = str).reshape(-1,1)
poslabels = np.full(np.shape(positive),1)

negset = np.concatenate((negative,neglabels), axis=1)
neutset = np.concatenate((neutral,neutlabels), axis=1)
posset =  np.concatenate((positive,poslabels), axis=1)

dataset = np.concatenate((negset,neutset,posset), axis=0)

import random
random.shuffle(dataset)

array([['How unhappy  some dogs like it though', '-1'],
       ['How unhappy  some dogs like it though', '-1'],
       ["Does anybody know if the Rand's likely to fall against the dollar? I got some money  I need to change into R but it keeps getting stronger unhappy ",
        '-1'],
       ...,
       [' and took aol instant messanger at money in and more. ', '0'],
       ["isn't it terrible that we live in a world where we have to address anticipated criticism of our thoughts first? happy",
        '1'],
       ["I'll be as strong as I can but it's hard unhappy  xxx", '-1']],
      dtype='<U161')

In [46]:
## Convert sentences to circuits w lambeq
parser = BobcatParser(root_cats=('NP','N'),verbose = 'text')

raw_train_diagrams = parser.sentences2diagrams(train,suppress_exceptions=True)
raw_val_diagrams = parser.sentences2diagrams(test,  suppress_exceptions=True)

ValueError: `tokenised` set to `False`, but variable `sentences` does not have type `List[str]`.